<h2>Import libraries<h2>

<h2>This notebook will be mainly used for the capstone project</h2>

In [112]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [113]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


<h3>Request Wikiedia page URL <h3>

In [114]:
url = 'http://en.turkcewiki.org/wiki/List_of_postal_codes_of_Canada:_M'
html_data = requests.get(url).text

In [115]:
# Create a beautiful soup obect
beautiful_soup = BeautifulSoup(html_data, 'html5lib')

<h3>Select desired table<h3>

In [116]:
# 
tables = beautiful_soup.find_all('table')

In [117]:
len(tables)

3

In [118]:
table_index = None
for index,table in enumerate(tables):
    if ("M3A" in str(table)):
        table_index = index

In [119]:
table_index

0

In [120]:
table = tables[table_index]


<h3>Extract data, reprocess it, and create our dataframe<h3>

In [121]:
table_contents=[]
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
neighborhood_wiki_df=pd.DataFrame(table_contents)

In [123]:
neighborhood_wiki_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


<h3>Sort the neighborhood dataframe in ascending order to match the Geospatial Coordinates dataframe and reset the index<h3>

In [124]:
neighborhood_df = neighborhood_wiki_df.sort_values(by='PostalCode', ascending=True)
neighborhood_df.head()

,PostalCode,Borough,Neighborhood
6,M1B,Scarborough,"Malvern, Rouge"
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
18,M1E,Scarborough,"Guildwood, Morningside, West Hill"
22,M1G,Scarborough,Woburn
26,M1H,Scarborough,Cedarbrae


In [125]:
neighborhood_df = neighborhood_df.reset_index( drop=True)
neighborhood_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [126]:
print(neighborhood_df.shape)

(103, 3)


<h3>Get the latitude and the longitude coordinates of each neighborhood<h3>

In [129]:
coord_url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv'
coordinates_csv_df = pd.read_csv(coord_url)
coordinates_csv_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [130]:
coordinates_df = coordinates_csv_df.drop(columns=['Postal Code'])
coordinates_df.head()

,Latitude,Longitude
0,43.806686,-79.194353
1,43.784535,-79.160497
2,43.763573,-79.188711
3,43.770992,-79.216917
4,43.773136,-79.239476


<h3>concatenate both dataframes<h3>

In [131]:
df = pd.concat([neighborhood_df,coordinates_df], axis=1)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [132]:
print(df.shape)

(103, 5)
